In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
from skimage.metrics import peak_signal_noise_ratio, structural_similarity
import numpy as np
import random
import glob
import os

In [ ]:
paths = glob.glob(os.path.join('val2017','*'))
print(len(paths))

In [ ]:
train_paths = paths[:4500]
val_paths = paths[4500:]

In [ ]:
def rgb2gray(rgb):
    r, g, b = rgb[:,:,0], rgb[:,:,1], rgb[:,:,2]
    gray = 0.2989 * r + 0.5870 * g + 0.1140 * b
    return gray

In [ ]:
def add_speckle(s1, looks = 4):
    # Numpy Gamma Distribution is defined in the shape-scale form
    # Mean 1 Var 1/looks
    gamma_shape = looks
    gamma_scale = 1/looks

    noise = np.random.gamma(gamma_shape, 
                            gamma_scale, 
                            (s1.shape[0],s1.shape[1]))
    
    s1 = s1 * np.clip(noise, 0.0, 1.0)
    return s1, noise

def add_noise(s1, var = 0.2):
    row,col = s1.shape
    mean = 0
    sigma = var**0.5
    gauss = np.random.normal(mean,sigma,(row,col))
    gauss = gauss.reshape(row,col)
    noisy = s1 + gauss
    noisy = np.clip(noisy, 0.0, 1.0)
    
    return noisy, gauss

In [ ]:
def norm(img):
    return (img - img.min())/(img.max() - img.min())

In [ ]:
def enl(img):
    return (np.nanmean(img)**2)/(np.nanstd(img)**2)

In [ ]:
def dataL(paths, batch_size, shape, var = 1, t = 'speckle', shuffle = True):
    
    batch_in  = np.zeros((batch_size, shape, shape, 1))
    batch_out = np.zeros((batch_size, shape, shape, 1))
    if shuffle:
        random.shuffle(paths)
    counter = 0
    
    while True:
        
        if counter>= len(paths) - batch_size:
            counter = 0
            if shuffle:
                random.shuffle(paths)
            
        for i in range(batch_size):
            img = plt.imread(paths[counter])[:shape, :shape, :]
            img = img/img.max()
            
            batch_out[i,:,:,0] = rgb2gray(img)
            if t=='speckle':
                batch_in[i,:,:,0], _  = add_speckle(batch_out[i, :,:,0], looks = var)
            elif t == 'gaussian':
                batch_in[i,:,:,0], _  = add_noise(batch_out[i, :,:,0], var = var)
                        
            counter += 1
            
        yield  np.clip(batch_in, 0.0, 1.0), np.clip(batch_out, 0.0, 1.0)

In [ ]:
train_gen = next(iter(dataL(train_paths, 1, shape, var=4)))
fig, axes = plt.subplots(nrows=1, ncols = 4, figsize = (20,5))
axes[0].imshow(train_gen[1][0,...,0])
img, _ = add_speckle(train_gen[1][0,...,0], looks = 1)
img = np.clip(img, 0.0, 1.0)
axes[1].imshow(img)
img, _ = add_speckle(train_gen[1][0,...,0], looks = 2)
img = np.clip(img, 0.0, 1.0)
axes[2].imshow(img)
img, _ = add_speckle(train_gen[1][0,...,0], looks = 4)
img = np.clip(img, 0.0, 1.0)
axes[3].imshow(img)

fig.tight_layout()
plt.show()

# Train-Test models

In [125]:
from Model import CNNSpeckleFilter
N_LAYER = 30
shape = 256
epochs = 60
batch_size = 12
varias = [1,2,4]# or [0.01, 0.02, 0.05, 0.08, 0.1]
train = False
noise = 'speckle' #or gaussian

for var in varias:
    # Train
    speckle_filter = CNNSpeckleFilter(input_shape=(shape,shape,1), n_layers=N_LAYER)
    
    if train:
        train_gen = dataL(train_paths, batch_size, shape, var=var, t = noise)
        val_gen = dataL(val_paths, batch_size, shape, var=var, t = noise)
        train_step = len(train_paths)//batch_size
        val_step = len(val_paths)//batch_size

        history = speckle_filter.train_model(epochs, train_gen, val_gen, train_step, val_step)
        speckle_filter.model.save_weights('weights/noSAR_model_'+str(N_LAYER)+'layers_ssim_mse_clipin_tanh_v3_'+str(noise)+'-var_'+str(var)+'.h5')
    else: 
        speckle_filter.model.load_weights('weights/noSAR_model_'+str(N_LAYER)+'layers_ssim_mse_clipin_tanh_v3_'+str(noise)+'-var_'+str(var)+'.h5')
    
    ## Test in any case
    test_paths = glob.glob(os.path.join('coco_test_dataset','*'))
    test_paths.sort()
    batch_size = len(test_paths)

    val_gen = dataL(test_paths, batch_size, shape, var = var, t = noise, shuffle = False)
    x,y = next(iter(val_gen))
    yp = speckle_filter.model.predict(x)

    for i in range(batch_size):

        fig, axes = plt.subplots(nrows=1,ncols=5, figsize = (30,10))
        axes[0].imshow(x[i,:,:,0], cmap = 'gray')
        axes[0].axis(False)

        axes[1].imshow(y[i,:,:,0], vmax = 1, cmap = 'gray')
        axes[1].axis(False)

        n = (yp[i,:,:,0] - yp[i,:,:,0].min()) / (yp[i,:,:,0].max() - yp[i,:,:,0].min())
        axes[2].imshow(n, vmax = 1, cmap = 'gray')
        axes[2].axis(False)

        axes[3].imshow(yp[i,:,:,0], vmax = 1, cmap = 'gray')
        axes[3].axis(False)

        axes[4].imshow(np.abs(x[i,:,:,0] - yp[i,:,:,0]), vmax = 1, cmap = 'gray')
        axes[4].axis(False)
        #plt.show()
        plt.close()

        plt.imsave('results_{}\\b_{}_noise-l_{}.png'.format(noise,i,var), x[i,:,:,0], cmap = 'gray')
        plt.imsave('results_{}\\b_{}_gt-l_{}.png'.format(noise,i,var), y[i,:,:,0], cmap = 'gray')
        plt.imsave('results_{}\\b_{}_pred-l_{}.png'.format(noise,i,var), n, cmap = 'gray')
        print('Looks {}'.format(var))
        print('PSNR(GT,N): ', peak_signal_noise_ratio(y[i,:,:,0], x[i,:,:,0]))
        print('PSNR(GT,P): ', peak_signal_noise_ratio(y[i,:,:,0], n))
              
        print('SSIM(GT,N): ', structural_similarity(y[i,:,:,0], x[i,:,:,0]))
        print('SSIM(GT,P): ', structural_similarity(y[i,:,:,0], n))
        
        print('ENL(N): ', 10*enl(x[i,:,:,0]))
        print('ENL(GT): ', 10*enl(y[i,:,:,0]))
        print('ENL(P): ', 10*enl(n))

Looks 1
PSNR(GT,N):  10.061862083260442
PSNR(GT,P):  20.58763857594783
SSIM(GT,N):  0.48111437869624485
SSIM(GT,P):  0.7740527077945484
ENL(N):  9.388087094496507
ENL(GT):  17.662411963411095
ENL(P):  18.36062619487039
Looks 1
PSNR(GT,N):  10.571781759743846
PSNR(GT,P):  25.165358969152507
SSIM(GT,N):  0.3210281722001462
SSIM(GT,P):  0.9426498477899382
ENL(N):  15.985020206105425
ENL(GT):  43.73408942494629
ENL(P):  37.79695218820845
Looks 1


C:\Users\aless\AppData\Local\Temp/ipykernel_21276/251059505.py:60: UserWarning: Inputs have mismatched dtype.  Setting data_range based on image_true.
  print('PSNR(GT,P): ', peak_signal_noise_ratio(y[i,:,:,0], n))


PSNR(GT,N):  11.132979889365567
PSNR(GT,P):  26.137634543336322
SSIM(GT,N):  0.32911485887361996
SSIM(GT,P):  0.8990596806000042
ENL(N):  13.2162926266046
ENL(GT):  30.64010748245701
ENL(P):  29.761395254264563
Looks 1
PSNR(GT,N):  12.819521338682874
PSNR(GT,P):  28.189066109175624
SSIM(GT,N):  0.3749165259128601
SSIM(GT,P):  0.9621658527368119
ENL(N):  10.05346906007587
ENL(GT):  19.481350099898933
ENL(P):  19.823218894845375
Looks 1
PSNR(GT,N):  13.213151773597364
PSNR(GT,P):  30.435376462189296
SSIM(GT,N):  0.30926321857137257
SSIM(GT,P):  0.972733837659089
ENL(N):  13.477317012959709
ENL(GT):  31.21746428907666
ENL(P):  28.95073285092632
Looks 1
PSNR(GT,N):  8.274762904912965
PSNR(GT,P):  27.292631108598613
SSIM(GT,N):  0.06365516256437717
SSIM(GT,P):  0.9783097473225709
ENL(N):  28.83910408916615
ENL(GT):  581.7165372010663
ENL(P):  386.79408651429435
Looks 1
PSNR(GT,N):  9.203929323969547
PSNR(GT,P):  23.191136432275474
SSIM(GT,N):  0.2103984160540494
SSIM(GT,P):  0.9471293796712

In [ ]:
from Model import CNNSpeckleFilter
N_LAYER = 30
shape = 256
epochs = 60
batch_size = 12
varias = [0.01, 0.02, 0.05, 0.08, 0.1]
train = False
noise = 'gaussian' #or gaussian

for var in varias:
    # Train
    speckle_filter = CNNSpeckleFilter(input_shape=(shape,shape,1), n_layers=N_LAYER)
    
    if train:
        train_gen = dataL(train_paths, batch_size, shape, var=var, t = noise)
        val_gen = dataL(val_paths, batch_size, shape, var=var, t = noise)
        train_step = len(train_paths)//batch_size
        val_step = len(val_paths)//batch_size

        history = speckle_filter.train_model(epochs, train_gen, val_gen, train_step, val_step)
        speckle_filter.model.save_weights('weights/noSAR_model_'+str(N_LAYER)+'layers_ssim_mse_clipin_tanh_v3_'+str(noise)+'-var_'+str(var)+'.h5')
    else: 
        speckle_filter.model.load_weights('weights/noSAR_model_'+str(N_LAYER)+'layers_ssim_mse_clipin_tanh_v3_'+str(noise)+'-var_'+str(var)+'.h5')
    
    ## Test in any case
    test_paths = glob.glob(os.path.join('coco_test_dataset','*'))
    test_paths.sort()
    batch_size = len(test_paths)

    val_gen = dataL(test_paths, batch_size, shape, var = var, t = noise, shuffle = False)
    x,y = next(iter(val_gen))
    yp = speckle_filter.model.predict(x)

    for i in range(batch_size):

        fig, axes = plt.subplots(nrows=1,ncols=5, figsize = (30,10))
        axes[0].imshow(x[i,:,:,0], cmap = 'gray')
        axes[0].axis(False)

        axes[1].imshow(y[i,:,:,0], vmax = 1, cmap = 'gray')
        axes[1].axis(False)

        n = (yp[i,:,:,0] - yp[i,:,:,0].min()) / (yp[i,:,:,0].max() - yp[i,:,:,0].min())
        axes[2].imshow(n, vmax = 1, cmap = 'gray')
        axes[2].axis(False)

        axes[3].imshow(yp[i,:,:,0], vmax = 1, cmap = 'gray')
        axes[3].axis(False)

        axes[4].imshow(np.abs(x[i,:,:,0] - yp[i,:,:,0]), vmax = 1, cmap = 'gray')
        axes[4].axis(False)
        #plt.show()
        plt.close()

        plt.imsave('results_{}\\b_{}_noise-l_{}.png'.format(noise,i,var), x[i,:,:,0], cmap = 'gray')
        plt.imsave('results_{}\\b_{}_gt-l_{}.png'.format(noise,i,var), y[i,:,:,0], cmap = 'gray')
        plt.imsave('results_{}\\b_{}_pred-l_{}.png'.format(noise,i,var), n, cmap = 'gray')
        print('Looks {}'.format(var))
        print('PSNR(GT,N): ', peak_signal_noise_ratio(y[i,:,:,0], x[i,:,:,0]))
        print('PSNR(GT,P): ', peak_signal_noise_ratio(y[i,:,:,0], n))
              
        print('SSIM(GT,N): ', structural_similarity(y[i,:,:,0], x[i,:,:,0]))
        print('SSIM(GT,P): ', structural_similarity(y[i,:,:,0], n))
        
        print('ENL(N): ', 10*enl(x[i,:,:,0]))
        print('ENL(GT): ', 10*enl(y[i,:,:,0]))
        print('ENL(P): ', 10*enl(n))